In [2]:
import os
import sys
from langchain_community.document_loaders import WebBaseLoader
from langchain_openai import AzureChatOpenAI
import re 
from langchain_community.graphs import Neo4jGraph
import dotenv
from langchain_community.graphs import Neo4jGraph
from langchain_core.documents import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from pathlib import Path
import numpy as np 
import pandas as pd
# from project import web_crawl
import subprocess
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [3]:
# list of top Tech leaders
url_list=[  
    'https://en.wikipedia.org/wiki/Elon_Musk',
    'https://en.wikipedia.org/wiki/Mark_Zuckerberg',
    'https://en.wikipedia.org/wiki/Bill_Gates',
    'https://en.wikipedia.org/wiki/Jeff_Bezos',
    'https://en.wikipedia.org/wiki/Steve_Jobs',
    'https://en.wikipedia.org/wiki/Sam_Altman',
    'https://en.wikipedia.org/wiki/Larry_Ellison',
    'https://en.wikipedia.org/wiki/Larry_Page',
    'https://en.wikipedia.org/wiki/Sundar_Pichai',
    'https://en.wikipedia.org/wiki/Satya_Nadella' 
    
]

In [4]:
# define a function to clean the extracted web URL data
import re #for regular expression 

def clean_text(text):
    # Remove HTML tags
    text = re.sub(r'<[^>]*?>', '', text)
    # Remove URLs
    text = re.sub(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', '', text)
    # Remove special characters
    text = re.sub(r'[^a-zA-Z0-9 ]', '', text)
    # Replace multiple spaces with a single space
    text = re.sub(r'\s{2,}', ' ', text)
    # Trim leading and trailing whitespace
    text = text.strip()
    # Remove extra whitespace
    text = ' '.join(text.split())
    return text

In [5]:
# extract the data from the URLs
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.documents import Document
from langchain_community.document_loaders import WebBaseLoader


def extract_data_from_URL(url):
    loader=WebBaseLoader([url])
    data=loader.load().pop().page_content
    data=clean_text(data)
    documents=[Document(page_content=data)]
    # print(documents)
    splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=100)
    smaller_doc=splitter.split_documents(documents)
    #print(len(smaller_doc))
    return smaller_doc

In [6]:
import dotenv 
from langchain_openai import AzureChatOpenAI
from openai import AzureOpenAI
import os  

dotenv.load_dotenv()
llm=AzureOpenAI(
        api_key=os.getenv('AZURE_OPENAI_API_KEY_2'),
        azure_endpoint=os.getenv('AZURE_OpenAI_ENDPOINT_2'),
        api_version=os.getenv('AZURE_OpenAI_API_VERSION_2')
)

In [27]:
for url in url_list:
    with open(f'../input/{url.split("/")[-1]}.txt','w') as file:
        smaller_doc=extract_data_from_URL(url)
        for doc in smaller_doc:
            file.write(str(doc.page_content))

276
100
150
132
151
41
62
85
36
32


In [22]:
# create graph 
import subprocess 

commands=['graphrag','index',"--root",'.']

try:
    results=subprocess.run(commands,capture_output=True,text=True,check=True)
    # print the output 
    print('Command executed successfully')
    print('output')
    print(results.stdout)
except subprocess.CalledProcessError as e:
    # handle error 
    print('error executing the command')
    print(e.stderr)


error executing the command
┌───────────────────── Traceback (most recent call last) ─────────────────────┐
│ D:\pythonProjects\KAG_Testing\graphragvenv\lib\site-packages\graphrag\cli\m │
│ ain.py:178 in _index_cli                                                    │
│                                                                             │
│   175 │   """Build a knowledge graph index."""                              │
│   176 │   from graphrag.cli.index import index_cli                          │
│   177 │                                                                     │
│ > 178 │   index_cli(                                                        │
│   179 │   │   root_dir=root,                                                │
│   180 │   │   verbose=verbose,                                              │
│   181 │   │   resume=resume,                                                │
│                                                                             │
│ ┌─────────

In [23]:
#Define the command as a list of arguments
command = [
    "python", "-m", "graphrag", "query",
    "--method", "global",
    "--query", "who are some famous people interacted with to elon musk, provide their names"
]

try:
    # Run the command and capture the output
    result = subprocess.run(command, capture_output=True, text=True, check=True)
    print("Command executed successfully.")
    print("Output:")
    print(result.stdout)  # Print the output of the command
except subprocess.CalledProcessError as e:
    # Handle errors
    print("Error executing the command:")
    print(e.stderr)  # Print the error message


Error executing the command:
┌───────────────────── Traceback (most recent call last) ─────────────────────┐
│ d:\pythonProjects\KAG_Testing\graphragvenv\lib\site-packages\graphrag\cli\m │
│ ain.py:447 in _query_cli                                                    │
│                                                                             │
│   444 │   │   │   │   query=query,                                          │
│   445 │   │   │   )                                                         │
│   446 │   │   case SearchType.GLOBAL:                                       │
│ > 447 │   │   │   run_global_search(                                        │
│   448 │   │   │   │   config_filepath=config,                               │
│   449 │   │   │   │   data_dir=data,                                        │
│   450 │   │   │   │   root_dir=root,                                        │
│                                                                             │
│ ┌────────